In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [ ]:
# Select only the columns you need here; removed debtor information for this example as it's not needed
cols = [
    "loan_id",
    "status",
    "client_id",
    "debtor_country",
    "trust_id",
    "amount",
    "created_at",
    "accepted_at",
    "refused_at",
    "repaid_at",
    "due_date",
    "insurance_status"
]

# Load data stored on DB files folder, exported from Metabase without uneeded columns
# Figures weren't matching due to the fact numbers in france are not separated with , and the decimal value is not .
df = pd.read_csv('DB files/loans.csv',
                 decimal='.',
                 thousands=',')

# Create a list of the dates to ensure they're converted as dates
date_cols = ["accepted_at", "created_at", "refused_at", "repaid_at", "due_date"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

# Convert amount to int
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
df.head()

In [ ]:
print("\nData Types and Nulls:")
df.info()

In [ ]:
print("\nDescriptive Statistics:")
df.describe()

In [ ]:
print("\nMissing Values:")
df.isna().sum()

#### Key findings ###
Accepted at has null values, meaning not all loans are accepted
No null loan id, meaning data quality is apparently ok
No null client id, debtor id, trust id, meaning joins against target tables are possible
Filtering by Country, status, insurance status are possible

In [ ]:
# create function for time filter
def filter_by_date(df, start=None, end=None, date_col="accepted_at"):
    if start:
        df = df[df[date_col] >= start]
    if end:
        df = df[df[date_col] <= end]
    return df

In [ ]:
# Question 1 - Monthly Production
# List of invalid status
invalid_status = ['REFUSED','EXPIRED','REJECTED','TIME_BARRED']

# Create new df filtered by valid status above
q1_df = df[~df["insurance_status"].isin(invalid_status)].copy()


# Filtered df with date function above; CHANGE DATES HERE
q1_df = filter_by_date(
    q1_df,
    start="2025-01-01",
    end="2025-12-31",
    date_col="accepted_at"
)

# Convert date to a monthly period, ie 2025-01-16 is converted to 2025-01-01
q1_df["month"] = q1_df["accepted_at"].dt.to_period("M").dt.to_timestamp()

# Group by month and sum amount
monthly_production = (
    q1_df
    .groupby("month")["amount"]
    .sum()
    .reset_index(name="monthly_production")
)

In [ ]:
# Line chart for question 1
plt.figure(figsize=(12,5))
sns.barplot(
    data=monthly_production,
    x="month",
    y="monthly_production"
)
plt.title("Monthly Production")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Question 2 - Average insurance rate per country per month
# List of valid insured status (WHERE insurance_status IN ('CLAIMED','SETTLED','ACTIVATED'))
insured_status = ['CLAIMED','SETTLED','ACTIVATED']

q2_df = df[df["insurance_status"].isin(insured_status)].copy()
q2_df["month"] = q2_df["accepted_at"].dt.to_period("M").dt.to_timestamp()

In [ ]:
# Chart for question 2 assist
def plot_avg_insurance_rate(
    data,
    country=None,
    start_date=None,
    end_date=None,
    chart_type="line"
):
    d = data.copy()

    # Filter by country
    if country:
        d = d[d["debtor_country"] == country]

    # Filter by date
    if start_date:
        d = d[d["month"] >= pd.to_datetime(start_date)]
    if end_date:
        d = d[d["month"] <= pd.to_datetime(end_date)]

    grouped = (
        d.groupby(["debtor_country", "month"])["amount"]
        .mean()
        .reset_index(name="avg_insurance_rate")
    )

    plt.figure(figsize=(12,5))

    if chart_type == "bar":
        sns.barplot(data=grouped, x="month", y="avg_insurance_rate")
    else:
        sns.lineplot(data=grouped, x="month", y="avg_insurance_rate")

    title = "Average Insurance Rate"
    if country:
        title += f" – {country}"

    plt.title(title)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Chart for question 2
# Country + timeframe, SELECT THE desired country and dates here
plot_avg_insurance_rate(
    q2_df,
    country="FR",
    start_date="2025-01-01",
    end_date="2025-08-31"
)

In [ ]:
# Question 3 - Metrics per trust
q3_df = df.copy()
q3_df["trust"] = q3_df["trust_id"].str[-5:]

summary = (
    q3_df
    .groupby("trust")
    .agg(
        total_outstanding=("amount", lambda x: x[q3_df.loc[x.index, "status"]
                                                .isin(["PAST_DUE","TO_REPAY"])].sum()),
        avg_loan_size=("amount", "mean"),
        insurance_rate=("amount", lambda x: x[q3_df.loc[x.index, "insurance_status"] == "SETTLED"].mean()),
        pct_non_french=("debtor_country", lambda x: (x != "FR").mean() * 100)
    )
    .reset_index()
)

In [ ]:
# Chart for question 3
fig, axes = plt.subplots(1, 4, figsize=(20,5))

# 1. Total outstanding
axes[0].bar(summary["trust"], summary["total_outstanding"])
axes[0].set_title("Total Outstanding")
axes[0].tick_params(axis='x', rotation=45)

# 2. Average loan size
axes[1].bar(summary["trust"], summary["avg_loan_size"])
axes[1].set_title("Average Loan Size")
axes[1].tick_params(axis='x', rotation=45)

# 3. Insurance rate
axes[2].bar(summary["trust"], summary["insurance_rate"])
axes[2].set_title("Insurance Rate")
axes[2].tick_params(axis='x', rotation=45)

# 4. % non-French loans
axes[3].bar(summary["trust"], summary["pct_non_french"])
axes[3].set_title("% Non-French Loans")
axes[3].tick_params(axis='x', rotation=45)

plt.suptitle("Loan Metrics per Trust", fontsize=16)
plt.tight_layout()
plt.show()